## Exercício 2

Um dataset contido no TensorFlow é o CelebA, um conjunto de cerca de 200,000 imágens de rostos de
celebridades. Cada imágem contem 40 características binárias, tais como sexo e idade (young, old).

Construa uma CNN que seja capaz de classificar o sexo da pessoa na imagem. Para servir de referência, um
bom classificador consegue perto de 95% no conjunto de testes.

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import cv2    
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils
from keras.optimizers import SGD

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model

from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

plt.style.use('ggplot')

%matplotlib inline

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Exploração dos dados

Será utilizado o banco de dados CelebA, que é composto por imagens de 178 x 218 px. Abaixo exemplos de como as figuras se parecem.

In [ ]:
# iniciando variáveis
main_folder = '../input/celeba-dataset/'
images_folder = main_folder + 'img_align_celeba/img_align_celeba/'

EXAMPLE_PIC = images_folder + '000087.jpg'

TRAINING_SAMPLES = 5000
VALIDATION_SAMPLES = 200
TEST_SAMPLES = 200
IMG_WIDTH = 178
IMG_HEIGHT = 218
BATCH_SIZE = 16
NUM_EPOCHS = 20

### Carregando os atributos de cada figura
Arquivo: list_attr_celeba.csv

In [ ]:
# Importando o banco que tem os atributos para cada figura
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

### Lista dos atributos disponíveis no banco CelebA
40 atributos

In [ ]:
# Lista de atributos disponíveis
for i, j in enumerate(df_attr.columns):
    print(i, j)

### Exemplo de uma imagem do CelebA
178 x 218 px

In [ ]:
# Mostrando uma figura e alguns atributos
img = load_img(EXAMPLE_PIC)
plt.grid(False)
plt.imshow(img)
df_attr.loc[EXAMPLE_PIC.split('/')[-1]][['Smiling','Male','Young']] #some attributes

### Distribuição do atributo sexo

In [ ]:
# Mulher ou Homem
plt.title('Mulher ou Homem')
sns.countplot(y='Male', data=df_attr, color="c")
plt.show()

## Passo 2: Separando o banco em treino, validação e teste

O particionamento recomendado do banco é:
* 1-162770 para treinamento
* 162771-182637 para validação
* 182638-202599 para teste

O arquivo com o particionamento está em <b>list_eval_partition.csv</b>

Para evitar um tempo de processamento muito elevado, será utilizado uma amostra com as seguitnes características:

* Training 10000 images
* Validation 2000 images
* Test 2000 Images


In [ ]:
# Particionamento recomendado
df_partition = pd.read_csv(main_folder + 'list_eval_partition.csv')
df_partition.head()

In [ ]:
# Mostrando o contador por partição
# 0 -> TRAINING
# 1 -> VALIDATION
# 2 -> TEST
df_partition['partition'].value_counts().sort_index()

#### Juntando o particionamento e o atributo na mesma base

In [ ]:
# Juntando o particionamento com o atributo
df_partition.set_index('image_id', inplace=True)
df_par_attr = df_partition.join(df_attr['Male'], how='inner')
df_par_attr.head()

### 2.1: Fazendo a separação (Treino, Validação, Teste)

O número de imagens precisa ser balanceado para garantir uma boa performance do modelo. Cada modelo terá sua própria pasta de treinamento, validação e teste de forma balanceada.

Este projeto de graduação explica como dados de treinamento desbalanceados impactam modelos CNN:

https://www.kth.se/social/files/588617ebf2765401cfcc478c/PHensmanDMasko_dkand15.pdf

Nesse passo são criadas funções que ajudarão a criar cada partição.

In [ ]:
def load_reshape_img(fname):
    img = load_img(fname)
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])

    # para treino e validação
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 218, 178, 3)
        y_ = df_[attr].values
    # para teste
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

## Passo 3: Pre-processando Imagens: Data Augmentation

Gerando Data Augmentation para as imagens.

Data Augmentation permite gerar imagens com modificações a prtir das originais. O modelo aprenderá com essas alterações (mudanças de angulo, tamanho e posição), podendo ser capaz de predizer melhor imagens nunca vistas mas que poderiam ter as mesmas variações de angulo, tamanho e posição.

### 3.1. Um exemplo de Data Augmentation:

Isso é como uma imagem se parece após data augmentation (baseado nos parametros escolhidos abaixo).

In [ ]:
# criando o gerador de imagem para data augmentation
datagen =  ImageDataGenerator(
  #preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

# carrega uma imagem e transforma
img = load_img(EXAMPLE_PIC)
x = img_to_array(img)/255.
x = x.reshape((1,) + x.shape)

# mostra 10 imagens alteradas da imagem carregada
plt.figure(figsize=(20,10))
plt.suptitle('Data Augmentation', fontsize=28)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.subplot(3, 5, i+1)
    plt.grid(False)
    plt.imshow( batch.reshape(218, 178, 3))
    
    if i == 9:
        break
    i += 1
    
plt.show()

O resultado é um novo conjunto de imagens modificadas que permitem que o modelo aprenda com essas variações para levar essas mudanças em consideração durante o processo de aprendizagem.

### 3.2. Construindo geradores de dados

In [ ]:
# dados de treino
x_train, y_train = generate_df(0, 'Male', TRAINING_SAMPLES)

# Treino - Preparação de dados - Data Augmentation com geradores
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=BATCH_SIZE,
)

In [ ]:
# dados de validação
x_valid, y_valid = generate_df(1, 'Male', VALIDATION_SAMPLES)

'''
# Validation - Data Preparation - Data Augmentation with generators
valid_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
)

valid_datagen.fit(x_valid)

validation_generator = valid_datagen.flow(
x_valid, y_valid,
)
'''

Com o gerador de dados criado e dados de validação carregados, estamos prontos para começar a modelagem.

## Passo 4: Construindo o modelo - Reconhecimento de sexo

A partir daqui sem utilizar o kernel copiado

### 4.1. Fazendo o modelo

In [ ]:
#Criando as camadas
i = Input(shape=x_train[0].shape)
x = Conv2D(32, (5, 5), strides = (2,2), activation='relu')(i)
x = Conv2D(64, (3, 3), strides = (2,2), activation='relu')(x)
x = Conv2D(128, (3, 3), strides = (2,2), activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation="softmax")(x)

model = Model(i, x)


'''# Import InceptionV3 Model
inc_model = InceptionV3(weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                        include_top=False,
                        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

print("number of layers:", len(inc_model.layers))
#inc_model.summary()'''

In [ ]:
#Compilando o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 4.2. Treinando o modelo

In [ ]:
r = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), steps_per_epoch= int(TRAINING_SAMPLES/BATCH_SIZE), epochs=NUM_EPOCHS)